<font size="6">Coursera capstone project - Hotels in Yangon</font>
<br><br>
<font size="3">Import libraries</font>

In [1]:
import requests  
# parse HTML / XML documents (Week 3)
from bs4 import BeautifulSoup 

import pandas as pd  
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json
# transform JSON file into Pandas
from pandas.io.json import json_normalize 

!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim  

     |████████████████████████████████| 102kB 5.9MB/s ta 0:00:011


<font size="3">Get list of townships</font>

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Townships_of_Yangon").text
soup = BeautifulSoup(data, 'html.parser')
nlist = []

for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    nlist.append(row.text)

ygn_df = pd.DataFrame({"Townships": nlist})
ygn_df.head(10)

,Townships
0,Ahlon Township
1,Bahan Township
2,Botataung Township
3,Dagon Seikkan Township
4,Dagon Township
5,Dala Township
6,Dawbon Township
7,East Dagon Township
8,Hlaing Township
9,Hlaingthaya Township


<font size="3">Get latitude & longtitude from list of townships</font>

In [3]:
def get_latlng(twsp):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Yangon, Myanmar'.format(twsp))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [4]:
coords = [get_latlng(twsp) for twsp in ygn_df["Townships"].tolist()]
coords

[[16.78806000000003, 96.12389000000007],
 [16.813150000000064, 96.15587000000005],
 [16.766670000000033, 96.16667000000007],
 [16.768490000000043, 96.16206000000005],
 [16.80528000000004, 96.15611000000007],
 [16.75917000000004, 96.15972000000005],
 [16.781940000000077, 96.18111000000005],
 [16.80528000000004, 96.15611000000007],
 [17.383330000000058, 95.83333000000005],
 [16.875630000000058, 96.06248000000005],
 [16.889170000000036, 96.10306000000008],
 [16.83000000000004, 96.12444000000005],
 [16.773030000000063, 96.16160000000008],
 [16.80583000000007, 96.12222000000008],
 [16.779810000000055, 96.14597000000003],
 [16.779580000000067, 96.15187000000003],
 [16.87000000000006, 96.13944000000004],
 [16.78776000000005, 96.17753000000005],
 [16.925000000000068, 96.09833000000003],
 [16.80528000000004, 96.15611000000007],
 [16.91303000000005, 96.16534000000007],
 [16.779440000000022, 96.15618000000006],
 [16.794440000000066, 96.17444000000006],
 [17.006940000000043, 96.08611000000008],
 [

In [5]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
ygn_df['Latitude'] = df_coords['Latitude']
ygn_df['Longitude'] = df_coords['Longitude']

print(ygn_df.shape)
ygn_df

(33, 3)


,Townships,Latitude,Longitude
0,Ahlon Township,16.78806,96.12389
1,Bahan Township,16.81315,96.15587
2,Botataung Township,16.76667,96.16667
3,Dagon Seikkan Township,16.76849,96.16206
4,Dagon Township,16.80528,96.15611
5,Dala Township,16.75917,96.15972
6,Dawbon Township,16.78194,96.18111
7,East Dagon Township,16.80528,96.15611
8,Hlaing Township,17.38333,95.83333
9,Hlaingthaya Township,16.87563,96.06248


In [6]:
ygn_df.to_csv("ygn_df.csv", index=False)

<font size="3">Import additional libraries for map visualization</font>

In [7]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium

     |████████████████████████████████| 102kB 10.2MB/s ta 0:00:01


<font size="3">Get geolocation and plot map of Yangon with Folium library</font>


In [8]:
address = 'Yangon, Myanmar'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The coordinates of Yangon, Myanmar is {}, {}.'.format(latitude, longitude))

The coordinates of Yangon, Myanmar is 16.7967129, 96.1609916.


In [9]:
map_ygn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, twsp in zip(ygn_df['Latitude'], ygn_df['Longitude'], ygn_df['Townships']):
    label = '{}'.format(twsp)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ygn)  
    
map_ygn

In [10]:
map_ygn.save('map_ygn.html')

<font size="3">Connect with Foursquare API</font>

In [16]:
CLIENT_ID = 'BJ3R0NDE5FJEH0QS2U312ZLJBASZJJ50KWQZRCCV02KFL4YY'
CLIENT_SECRET = 'OZOW1V52JBASEHSZFYZ5RI1OG3DL2KX5DMPTNIAB2Y1NBEVO'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + 'Printed')
print('CLIENT_SECRET:' + 'Printed')

Your credentails:
CLIENT_ID: Printed
CLIENT_SECRET:Printed


<font size="3">From API, obtain top 100 venues that are within a radius of 1000 meters.</font>

In [17]:
radius = 1000
LIMIT = 100
places = []

for lat, long, twsp in zip(ygn_df['Latitude'], ygn_df['Longitude'], ygn_df['Townships']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        places.append((
            twsp,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
places_df = pd.DataFrame(places)

# define the column names
places_df.columns = ['Township', 'Latitude', 'Longitude', 'Name', 'Latitude', 'Longitude', 'Category']

print(places_df.shape)
places_df.head(20)

(791, 7)


,Township,Latitude,Longitude,Name,Latitude,Longitude,Category
0,Ahlon Township,16.78806,96.12389,Coca Suki,16.790966,96.123282,Asian Restaurant
1,Ahlon Township,16.78806,96.12389,Parisian,16.795493,96.126335,Bakery
2,Ahlon Township,16.78806,96.12389,Lotaya Restaurant,16.788886,96.127490,Restaurant
3,Ahlon Township,16.78806,96.12389,Apple Hotpot & Dim Sum,16.786116,96.130737,Dim Sum Restaurant
4,Ahlon Township,16.78806,96.12389,Ahlone Railway Station,16.790524,96.131020,Platform
5,Ahlon Township,16.78806,96.12389,Pan Swel Taw,16.790233,96.131541,Restaurant
6,Ahlon Township,16.78806,96.12389,ေရြွနန္းေတာ္ မုန့္တီ,16.784361,96.132085,Noodle House
7,Bahan Township,16.81315,96.15587,Real Fitness Gym,16.816024,96.153980,Athletics & Sports
8,Bahan Township,16.81315,96.15587,Golden Duck (Sayar San Road),16.816230,96.157350,Asian Restaurant
9,Bahan Township,16.81315,96.15587,Manana,16.817201,96.156393,Burrito Place


In [23]:
#Check number of venues per township
places_df.groupby(["Township"]).count()

,Latitude,Longitude,Name,Latitude,Longitude,Category
Township,,,,,,
Ahlon Township,7,7,7,7,7,7
Bahan Township,62,62,62,62,62,62
Botataung Township,35,35,35,35,35,35
Dagon Seikkan Township,43,43,43,43,43,43
Dagon Township,43,43,43,43,43,43
Dala Township,2,2,2,2,2,2
Dawbon Township,4,4,4,4,4,4
East Dagon Township,43,43,43,43,43,43
Hlaingthaya Township,4,4,4,4,4,4


<font size="3">Return categories of all places from Foursquare API.</font>

In [24]:
#Find out how many unique categories from all venues
print('There are {} unique categories.'.format(len(places_df['Category'].unique())))

There are 97 unique categories.


In [25]:
# print out list of categories
places_df['Category'].unique()[:50]

array(['Asian Restaurant', 'Bakery', 'Restaurant', 'Dim Sum Restaurant',
       'Platform', 'Noodle House', 'Athletics & Sports', 'Burrito Place',
       'Korean Restaurant', 'Sandwich Place', 'Spa', 'Gym',
       'Shopping Mall', 'Steakhouse', 'Bar', 'Japanese Restaurant',
       'Diner', 'Pizza Place', 'Tea Room', 'Bubble Tea Shop',
       'Chinese Restaurant', 'Hotel', 'Coffee Shop', 'Ice Cream Shop',
       'Thai Restaurant', 'Beer Bar', 'Whisky Bar', 'BBQ Joint',
       'Italian Restaurant', 'American Restaurant', 'French Restaurant',
       'Burger Joint', 'Café', 'Pub', 'Multiplex', 'Bookstore',
       'Kids Store', 'Convenience Store', 'Plaza', 'Szechuan Restaurant',
       'Hostel', 'Gift Shop', 'Seafood Restaurant',
       'Other Great Outdoors', 'Burmese Restaurant', 'Sushi Restaurant',
       'Cruise', 'Deli / Bodega', 'Indian Restaurant', 'Ramen Restaurant'],
      dtype=object)

In [26]:
ygn_onehot = pd.get_dummies(places_df[['Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ygn_onehot['Townships'] = places_df['Township'] 

# move neighborhood column to the first column
fixed_columns = [ygn_onehot.columns[-1]] + list(ygn_onehot.columns[:-1])
ygn_onehot = ygn_onehot[fixed_columns]

print(ygn_onehot.shape)
ygn_onehot.head()

(791, 98)


,Townships,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boarding House,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cruise,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gift Shop,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Kids Store,Korean Restaurant,Lounge,Market,Motel,Movie Theater,Moving Target,Multiplex,Music Venue,Neighborhood,Noodle House,North Indian Restaurant,Other Great Outdoors,Park,Pizza Place,Platform,Plaza,Port,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Tourist Information Center,Train Station,Video Store,Whisky Bar
0,Ahlon Township,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ahlon Township,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ahlon Township,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ahlon Township,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ahlon Township,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
ygn_grouped = ygn_onehot.groupby(["Townships"]).mean().reset_index()

print(ygn_grouped.shape)
ygn_grouped

(29, 98)


,Townships,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boarding House,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cruise,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gift Shop,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Kids Store,Korean Restaurant,Lounge,Market,Motel,Movie Theater,Moving Target,Multiplex,Music Venue,Neighborhood,Noodle House,North Indian Restaurant,Other Great Outdoors,Park,Pizza Place,Platform,Plaza,Port,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Tourist Information Center,Train Station,Video Store,Whisky Bar
0,Ahlon Township,0.000000,0.142857,0.000000,0.00,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bahan Township,0.016129,0.032258,0.016129,0.00,0.032258,0.048387,0.048387,0.000000,0.016129,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.016129,0.000000,0.032258,0.000000,0.016129,0.000000,0.016129,0.032258,0.000000,0.000000,0.032258,0.0,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.064516,0.000000,0.000000,0.016129,0.000000,0.000000,0.032258,0.048387,0.016129,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.048387,0.000000,0.016129,0.000000,0.016129,0.000000,0.000000,0.016129,0.000000,0.016129,0.000000,0.000000,0.000000,0.048387,0.016129,0.000000,0.000000,0.016129,0.016129,0.016129,0.000000,0.000000,0.000000,0.016129
2,Botataung Township,0.000000,0.028571,0.000000,0.00,0.000000,0.000000,0.085714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.085714,0.000000,0.000000,0.085714,0.085714,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.028571,0.000000,0.028571,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057143,0.028571,0.000000,0.000000,0.000000,0.028571,0.114286,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.028571,0.028571,0.000000,0.028571,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.028571,0.028571,0.000000,0.000000,0.000000,0.000000
3,Dagon Seikkan Township,0.000000,0.023256,0.000000,0

In [46]:
#Group the total number of hotels within Yangon

len(ygn_grouped[ygn_grouped["Hotel"] > 0])

19

In [47]:
ygn_hotel = ygn_grouped[["Townships","Hotel"]]
ygn_hotel

,Townships,Hotel
0,Ahlon Township,0.000000
1,Bahan Township,0.064516
2,Botataung Township,0.114286
3,Dagon Seikkan Township,0.093023
4,Dagon Township,0.093023
5,Dala Township,0.000000
6,Dawbon Township,0.000000
7,East Dagon Township,0.093023
8,Hlaingthaya Township,0.000000
9,Insein Township,0.000000


In [48]:
kclusters = 5

ygn_clustering = ygn_hotel.drop(["Townships"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ygn_clustering)

kmeans.labels_[0:10]

array([3, 0, 1, 4, 4, 3, 3, 4, 3, 3], dtype=int32)

In [49]:
ygn_hotel2 = ygn_hotel.copy()
ygn_hotel2["Cluster Labels"] = kmeans.labels_
ygn_hotel2

,Townships,Hotel,Cluster Labels
0,Ahlon Township,0.000000,3
1,Bahan Township,0.064516,0
2,Botataung Township,0.114286,1
3,Dagon Seikkan Township,0.093023,4
4,Dagon Township,0.093023,4
5,Dala Township,0.000000,3
6,Dawbon Township,0.000000,3
7,East Dagon Township,0.093023,4
8,Hlaingthaya Township,0.000000,3
9,Insein Township,0.000000,3


In [50]:
#merge latitude & longitude data

ygn_hotel2 = ygn_hotel2.join(ygn_df.set_index("Townships"), on="Townships")

print(ygn_hotel2.shape)
ygn_hotel2.head(10) 

(29, 5)


,Townships,Hotel,Cluster Labels,Latitude,Longitude
0,Ahlon Township,0.000000,3,16.78806,96.12389
1,Bahan Township,0.064516,0,16.81315,96.15587
2,Botataung Township,0.114286,1,16.76667,96.16667
3,Dagon Seikkan Township,0.093023,4,16.76849,96.16206
4,Dagon Township,0.093023,4,16.80528,96.15611
5,Dala Township,0.000000,3,16.75917,96.15972
6,Dawbon Township,0.000000,3,16.78194,96.18111
7,East Dagon Township,0.093023,4,16.80528,96.15611
8,Hlaingthaya Township,0.000000,3,16.87563,96.06248
9,Insein Township,0.000000,3,16.88917,96.10306


In [51]:
# sort results by cluster Labels

ygn_hotel2.sort_values(["Cluster Labels"], inplace=True)
ygn_hotel2

,Townships,Hotel,Cluster Labels,Latitude,Longitude
1,Bahan Township,0.064516,0,16.81315,96.15587
25,Tamwe Township,0.047619,0,16.81000,96.17778
10,Kamayut Township,0.052632,0,16.83000,96.12444
14,Latha Township,0.116883,1,16.77958,96.15187
2,Botataung Township,0.114286,1,16.76667,96.16667
21,Pazundaung Township,0.137931,1,16.79444,96.17444
16,Mingala Taungnyunt Township,0.142857,1,16.78776,96.17753
13,Lanmadaw Township,0.131148,1,16.77981,96.14597
24,South Okkalapa Township,0.250000,2,16.85074,96.18494
26,Thaketa Township,0.000000,3,16.80412,96.21216


<font size="4">Visualize clusters</font>

In [52]:
import numpy as np 

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ygn_hotel2['Latitude'], ygn_hotel2['Longitude'], ygn_hotel2['Townships'], ygn_hotel2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<font size="4">Examine clusters</font>

<font size="2">Cluster 0</font>

In [53]:
ygn_hotel2.loc[ygn_hotel2['Cluster Labels'] == 0]

,Townships,Hotel,Cluster Labels,Latitude,Longitude
1,Bahan Township,0.064516,0,16.81315,96.15587
25,Tamwe Township,0.047619,0,16.81000,96.17778
10,Kamayut Township,0.052632,0,16.83000,96.12444


<font size="2">Cluster 1</font>

In [54]:
ygn_hotel2.loc[ygn_hotel2['Cluster Labels'] == 1]

,Townships,Hotel,Cluster Labels,Latitude,Longitude
14,Latha Township,0.116883,1,16.77958,96.15187
2,Botataung Township,0.114286,1,16.76667,96.16667
21,Pazundaung Township,0.137931,1,16.79444,96.17444
16,Mingala Taungnyunt Township,0.142857,1,16.78776,96.17753
13,Lanmadaw Township,0.131148,1,16.77981,96.14597


<font size="2">Cluster 2</font>

In [55]:
ygn_hotel2.loc[ygn_hotel2['Cluster Labels'] == 2]

,Townships,Hotel,Cluster Labels,Latitude,Longitude
24,South Okkalapa Township,0.25,2,16.85074,96.18494


<font size="2">Cluster 3</font>

In [56]:
ygn_hotel2.loc[ygn_hotel2['Cluster Labels'] == 3]

,Townships,Hotel,Cluster Labels,Latitude,Longitude
26,Thaketa Township,0.0,3,16.80412,96.21216
19,North Okkalapa Township,0.0,3,16.91303,96.16534
17,Mingaladon Township,0.0,3,16.92500,96.09833
27,Thingangyun Township,0.0,3,16.82778,96.19750
0,Ahlon Township,0.0,3,16.78806,96.12389
9,Insein Township,0.0,3,16.88917,96.10306
8,Hlaingthaya Township,0.0,3,16.87563,96.06248
6,Dawbon Township,0.0,3,16.78194,96.18111
5,Dala Township,0.0,3,16.75917,96.15972
12,Kyimyindaing Township,0.0,3,16.80583,96.12222
